# Different Tool Configuration Methods

This notebook demonstrates different approaches to configuring tools for LangChain agents, from simple lists to advanced error handling.

## Key Concepts
- **Method 1**: Simple list of tools (quick setup)
- **Method 2**: ToolNode with custom configuration (production-ready)
- **Error Handling**: Making agents robust to tool failures

## Why Tool Configuration Matters
- **Without error handling**: Errors crash the agent
- **With error handling**: Errors become feedback for the model
- **Model adaptation**: Agent can retry or adjust approach based on error messages

## Prerequisites

Make sure you have the required packages installed:

```bash
pip install langchain langchain-community langchain-core langgraph pydantic
ollama pull qwen3
ollama serve
```

In [1]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain.agents import create_agent, ToolNode
import tools

## Method 1: Simple List of Tools

The simplest approach - just pass tools as a list. LangChain handles everything automatically.

In [2]:
print("=== Method 1: Simple List of Tools ===")

# Create model instance
model = ChatOllama(model="qwen3")

# Method 1: Pass list of tools (simple approach)
agent1 = create_agent(
    model, 
    tools=[tools.web_search, tools.calculate]
)

print("✓ Agent 1 created with simple tool list")
print("  - LangChain automatically creates ToolNode")
print("  - Default error handling behavior")
print("  - Good for prototyping and simple use cases")

=== Method 1: Simple List of Tools ===
✓ Agent 1 created with simple tool list
  - LangChain automatically creates ToolNode
  - Default error handling behavior
  - Good for prototyping and simple use cases


## Method 2: Advanced ToolNode Configuration

Create a ToolNode explicitly with custom settings for better control over tool execution.

In [3]:
print("=== Method 2: Advanced ToolNode Configuration ===")

# Method 2: Use ToolNode with error handling (advanced approach)
tool_node = ToolNode(
    tools=[tools.web_search, tools.calculate],
    handle_tool_errors="Please check your input and try again. Error details will help you correct the issue."
)

agent2 = create_agent(model, tools=tool_node)

print("✓ Agent 2 created with advanced ToolNode")
print("  - Custom error handling messages")
print("  - Better control over tool execution")
print("  - Production-ready error recovery")

=== Method 2: Advanced ToolNode Configuration ===
✓ Agent 2 created with advanced ToolNode
  - Custom error handling messages
  - Better control over tool execution
  - Production-ready error recovery


## Comparing Both Approaches

Let's test both agents with the same query to see how they behave:

In [4]:
# Test query that uses multiple tools
test_query = "Search for Python tutorials and calculate 10 times 5"

print(f"Testing both agents with: '{test_query}'")
print("=" * 60)

# Test Agent 1 (Simple)
print("\n=== Agent 1 Results (Simple Tool List) ===")
try:
    result1 = agent1.invoke({"messages": test_query})
    print(f"✓ Response: {result1['messages'][-1].content}")
except Exception as e:
    print(f"✗ Error: {e}")

# Test Agent 2 (Advanced)
print("\n=== Agent 2 Results (Advanced ToolNode) ===")
try:
    result2 = agent2.invoke({"messages": test_query})
    print(f"✓ Response: {result2['messages'][-1].content}")
except Exception as e:
    print(f"✗ Error: {e}")

print("\n💡 Both agents work the same for valid inputs, but Agent 2 handles errors better")

Testing both agents with: 'Search for Python tutorials and calculate 10 times 5'

=== Agent 1 Results (Simple Tool List) ===
✓ Response: <think>
Okay, let me process what the user asked. They wanted two things: a search for Python tutorials and the calculation of 10 times 5. 

First, I called the web_search function with "Python tutorials" to get relevant resources. The results came back with five links, including some tutorials, projects, and communities. I formatted those results neatly, making sure each entry had a title, description, and URL.

Next, the calculation part. The user asked for 10 multiplied by 5. I used the calculate function with the expression "10 * 5". The result was 50, which I presented clearly.

Now, I need to combine both parts into a coherent response. I'll start with the search results, listing each item with the details from the tool response. Then, I'll add the calculation result at the end. I should make sure the information is easy to read, maybe using bul